In [28]:
import pandas as pd
import os 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import string
import matplotlib.pyplot as plt;

import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
text_variable='clean_text'
target_feature='category'
stop_words = stopwords.words('english') 

In [3]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"(http\S+|http)", "", txt) # remove links 
    txt = re.sub('[^a-zA-Z ]+', '', txt) #only allows for letters
    txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stop_words ]) # stem & remove stop words
    #made redudent by only accepting text
    '''
    txt = re.sub(r"(@\S+)", "", txt)  # remove hashtags
    txt = txt.translate(str.maketrans('', '', string.punctuation)) # remove punctuations 
 
    
    txt = ''.join([i for i in txt if not i.isdigit()]).strip() # remove digits ()
    '''
    return txt

In [4]:
directory_name='datasets/Twitter and Reddit Sentimental analysis Dataset'
# all_files = os.listdir(directory_name)   # imagine you're one directory above test dir
# txt_files = filter(lambda x: x[-4:] == '.txt', all_files)
# dictionary=[]

# for filename in txt_files:
# 	f = open(directory_name+filename,'r')
# 	dictionary.append({'ArticleId':filename[0:-4],'tweet_contents':f.read()})
# 	f.close()
# tweet_df = pd.DataFrame(dictionary)
data_df = pd.read_csv("datasets/Twitter and Reddit Sentimental analysis Dataset/Twitter_Data.csv") 
#cleaning type of data
data_df[text_variable] = data_df[text_variable].astype(str)
data_df[text_variable] = data_df[text_variable].apply(clean_text)  
# dropping nans and reindexing
data_df = data_df.dropna()
data_df=data_df.reset_index(drop=True)

data_df

,clean_text,category
0,modi promis minimum govern maximum govern expe...,-1.0
1,talk nonsens continu drama vote modi,0.0
2,say vote modi welcom bjp told rahul main camp...,1.0
3,ask support prefix chowkidar name modi great s...,1.0
4,answer among power world leader today trump pu...,1.0
...,...,...
162968,crore paid neerav modi recov congress leader ...,-1.0
162969,dear rss terrorist payal gawar modi kill plu ...,-1.0
162970,cover interact forum left,0.0
162971,big project came india modi dream project happ...,0.0


In [5]:
# def train_test_data(full_df):   
#     y = full_df[target_feature]   # define target and feature column
#     X = full_df[text_variable]
     
#     text_train, text_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42,stratify=y)  
#     vect = CountVectorizer(min_df=5, ngram_range=(1, 4)) # create Count vectorizer.
#     X_train = vect.fit(text_train).transform(text_train) # transform text_train  into a vector 
#     X_test = vect.transform(text_test) 
#     feature_names = vect.get_feature_names() # to return all words used in vectorizer
  
#     return X_train, X_test, y_train, y_test, feature_names
def train_test_crosval(train, test):
#     balanced_train = balance(train)
    y_train = train[target_feature]   
    text_train = train[text_variable]
    y_test = test[target_feature]   
    text_test = test[text_variable]
     
    #text_train, text_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42,stratify=y)  
    vect = TfidfVectorizer(min_df=5, ngram_range=(1, 4)) # create Count vectorizer.
    X_train = vect.fit(text_train).transform(text_train) # transform text_train  into a vector 
    X_test = vect.transform(text_test) 
    feature_names = vect.get_feature_names() # to return all words used in vectorizer
  
    return X_train, X_test, y_train, y_test, feature_names

In [6]:
def print_model_performance(model,X_train,X_test,y_train,y_test):
    training_sample = model.predict(X_train)
    testing_sample = model.predict(X_test)
    print('training ')
    #print(classification_report(training_sample, y_train))  #uncomment if you want to see full report 
    print('train accuracy ',accuracy_score(training_sample, y_train))
    #print('train precision_score ',precision_score(training_sample, y_train)) 
    #print('train recall score',recall_score(training_sample, y_train)) 
    
    print('\n testing  ')
    print(classification_report(testing_sample, y_test))   #uncomment if you want to see full report 
    print('test average accuracy ',accuracy_score(testing_sample, y_test))
    #print('test average precision_score ',precision_score(testing_sample, y_test)) 
    #print('test average recall score',recall_score(testing_sample, y_test)) 
    #print('test AUC ',roc_auc_score(testing_sample, y_test))
    
    print(confusion_matrix( y_test,testing_sample))

In [7]:



#y = id_tweets['Tone']   # define target and feature column
#X = id_tweets['cleaned_text']
to_dataset=data_df.copy()
#resample_sizes=[1000,1500,2000,2500]
# resample_sizes=[1000]
cv = KFold(n_splits=2, random_state=42, shuffle=False)
full_prediciton=[]
# for resample_size in resample_sizes:
for train_index, test_index in cv.split(to_dataset):
    train, test = to_dataset.loc[train_index], to_dataset.loc[test_index]

    X_train, X_test, y_train, y_test, feature_names=train_test_crosval(train,test)
    lgstc = LogisticRegressionCV()
    lgstc.fit(X_train, y_train)
    #print_model_performance(lgstc, X_train, X_test, y_train, y_test)
    prediction = lgstc.predict(X_test)
    full_prediciton.append(prediction)
predictions=[]

for set_of_prediction in full_prediciton:
    for predicted in set_of_prediction:
        predictions.append(predicted)


/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase 

In [20]:
# to_dataset.isnull().sum(axis = 0)
df_copy=to_dataset.copy()
df_copy

,clean_text,category
0,modi promis minimum govern maximum govern expe...,-1.0
1,talk nonsens continu drama vote modi,0.0
2,say vote modi welcom bjp told rahul main camp...,1.0
3,ask support prefix chowkidar name modi great s...,1.0
4,answer among power world leader today trump pu...,1.0
...,...,...
162968,crore paid neerav modi recov congress leader ...,-1.0
162969,dear rss terrorist payal gawar modi kill plu ...,-1.0
162970,cover interact forum left,0.0
162971,big project came india modi dream project happ...,0.0


In [21]:
print('outcome of training')
print(classification_report( df_copy[target_feature],predictions))   #uncomment if you want to see full report 
print('test average accuracy ',accuracy_score( df_copy[target_feature],predictions))
print(confusion_matrix( df_copy[target_feature],predictions))
df_copy['prediction']=predictions
# predictions.clear()
# full_prediciton.clear()
output_df=df_copy.drop(text_variable, axis=1)
# predictions.clear()
# full_prediciton.clear()

outcome of training


ValueError: Found input variables with inconsistent numbers of samples: [162973, 0]

In [29]:
df_copy['prediction']=output_df['prediction']

In [30]:
df_copy['polarity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.polarity)
df_copy['subjectivity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.subjectivity)

In [31]:

def sentiment_analyzer_scores(text, threshold=0.05, engl=True):
    analyser = SentimentIntensityAnalyzer()

    if engl:
        trans = text
    else:
        trans = translator.translate(text).text

    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
    if lb >= threshold:
        return 1
    elif (lb > -threshold) and (lb < threshold):
        return 0
    else:
        return -1

In [32]:
df_copy['vader_sentiment']=df_copy[text_variable].apply(lambda tweet: sentiment_analyzer_scores(tweet))

In [33]:
df_amalgamate=df_copy.copy()
df_amalgamate

,clean_text,category,prediction,polarity,subjectivity,vader_sentiment
0,modi promis minimum govern maximum govern expe...,-1.0,-1.0,-0.500000,1.000000,-0.3612
1,talk nonsens continu drama vote modi,0.0,0.0,0.000000,0.000000,0.0000
2,say vote modi welcom bjp told rahul main camp...,1.0,1.0,0.166667,0.333333,0.4404
3,ask support prefix chowkidar name modi great s...,1.0,-1.0,0.450000,0.566667,0.8555
4,answer among power world leader today trump pu...,1.0,1.0,0.000000,0.000000,0.0000
...,...,...,...,...,...,...
162968,crore paid neerav modi recov congress leader ...,-1.0,-1.0,-0.291667,0.541667,-0.1027
162969,dear rss terrorist payal gawar modi kill plu ...,-1.0,-1.0,0.000000,0.000000,-0.9062
162970,cover interact forum left,0.0,0.0,0.000000,0.000000,0.0000
162971,big project came india modi dream project happ...,0.0,0.0,0.000000,0.100000,0.2500


In [34]:
def train_test_crosval_prob(train,test, features):
#     balanced_train = balance(train)
    y_train = train[target_feature]   
    text_train = train[features]
    y_test = test[target_feature]   
    text_test = test[features]
    return X_train, X_test, y_train, y_test

In [35]:
cv = KFold(n_splits=2, random_state=42, shuffle=False)
full_prediciton=[]
features=['polarity','subjectivity','prediction','vader_sentiment']
# for resample_size in resample_sizes:
for train_index, test_index in cv.split(df_amalgamate):
    train, test = df_amalgamate.loc[train_index], df_amalgamate.loc[test_index]

    X_train, X_test, y_train, y_test=train_test_crosval_prob(train,test,features)
    lgstc = LogisticRegressionCV()
    lgstc.fit(X_train, y_train)
    #print_model_performance(lgstc, X_train, X_test, y_train, y_test)
    prediction = lgstc.predict(X_test)
    full_prediciton.append(prediction)
predictions=[]

for set_of_prediction in full_prediciton:
    for predicted in set_of_prediction:
        predictions.append(predicted)

/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [81487, 81486]

In [ ]:
print('outcome of training')
print(classification_report( df_copy[target_feature],predictions))   #uncomment if you want to see full report 
print('test average accuracy ',accuracy_score( df_copy[target_feature],predictions))
print(confusion_matrix( df_copy[target_feature],predictions))
df_copy['prediction']=predictions
# predictions.clear()
# full_prediciton.clear()
output_df=df_copy.drop(text_variable, axis=1)